In [ ]:
from Mnist_MoE.train_MoE import moe_train_loop, create_moe_model

moe = create_moe_model(
        nb_experts=7, 
        k_training=2, # Reduced k to force distinct specialization
        k_inference=1, 
        tau=1.5, # Higher temperature (>1) forces exploration of all experts
        lambda_balance=0.3 # Strong penalty for expert collapse
    )

# Routing policy: 'hard' (tok_K), 'soft' (sotfmax or gumbell_softmax), 'hybrid' (soft with a thresholding)
accuracies = {}
moe_train_info = moe_train_loop(moe, 
                                epochs=10, # Increased epochs as balancing takes time to converge
                                selection_policy_t='soft',
                                selection_policy_i='hybrid', 
                                gumbell_softmax_t=True,
                                gumbell_softmax_i=False, 
                                threshold=None)
accuracies.update(moe_train_info[0])


Starting MoE training...
Number of Experts: 7 | k_training: 2 | k_inference: 1

experts: Expert 1: MNIST_CNN2, Expert 2: MNIST_CNN2, Expert 3: MNIST_CNN2, Expert 4: MNIST_CNN2, Expert 5: MNIST_CNN2, Expert 6: MNIST_CNN2, Expert 7: MNIST_CNN2

Epoch [1/10] - Train Loss: 0.3502, Train Acc: 92.66% | Test Loss: 0.1309, Test Acc: 95.92%
Expert Repartition: ['0.084', '0.055', '0.054', '0.097', '0.095', '0.411', '0.203']
 Lambda Balance: 0.100000

Epoch [2/10] - Train Loss: 0.1893, Train Acc: 97.55% | Test Loss: 0.0964, Test Acc: 96.98%
Expert Repartition: ['0.034', '0.152', '0.168', '0.092', '0.184', '0.312', '0.057']
 Lambda Balance: 0.100000

Epoch [3/10] - Train Loss: 0.1689, Train Acc: 98.12% | Test Loss: 0.0707, Test Acc: 97.71%
Expert Repartition: ['0.009', '0.061', '0.416', '0.084', '0.027', '0.201', '0.202']
 Lambda Balance: 0.100000

Epoch [4/10] - Train Loss: 0.1606, Train Acc: 98.41% | Test Loss: 0.1319, Test Acc: 95.54%
Expert Repartition: ['0.179', '0.532', '0.000', '0.169', '0

In [2]:
import matplotlib.pyplot as plt
informations = moe_train_info[1]

plt.figure(figsize=(12, 5))
plt.plot([informations[epoch]["repartition"] for epoch in informations], label="Expert Repartition")
plt.xlabel("Epochs")
plt.title("Expert Repartition over Epochs")
plt.legend()
plt.show()

NameError: name 'moe_train_info' is not defined

In [ ]:
from Mnist_MoE.train_dense import train_dense_model

accuracies.update(train_dense_model())

In [ ]:
print("\nExpert Accuracies:")
for expert, acc in accuracies.items():
    print(f"{expert}: {acc:.2f}%")